Deep Learning
=============

Assignment 2
------------

Previously in `1_notmnist.ipynb`, we created a pickle with formatted datasets for training, development and testing on the [notMNIST dataset](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html).

The goal of this assignment is to progressively train deeper and more accurate models using TensorFlow.

In [42]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range
print("TensorFlow Version:",tf.__version__)

TensorFlow Version: 1.13.1


First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


We're first going to train a multinomial logistic regression using simple gradient descent.

TensorFlow works like this:
* First you describe the computation that you want to see performed: what the inputs, the variables, and the operations look like. These get created as nodes over a computation graph. This description is all contained within the block below:

      with graph.as_default():
          ...

* Then you can run the operations on this graph as many times as you want by calling `session.run()`, providing it outputs to fetch from the graph that get returned. This runtime operation is all contained in the block below:

      with tf.Session(graph=graph) as session:
          ...

Let's load all the data into TensorFlow and build the computation graph corresponding to our training:

In [38]:
log_dir = './logs/2_fullyconnected'
#Delete previous logs
if tf.gfile.Exists(log_dir):
    tf.gfile.DeleteRecursively(log_dir)
#Create folder
tf.gfile.MakeDirs(log_dir)

In [32]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 1000
valid_subset = valid_dataset.shape[0]
test_subset = test_dataset.shape[0]

# We can't initialize these variables to 0 - the network will get stuck.
def weight_variable(shape):
    """Create a weight variable with appropriate initialization."""
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    """Create a bias variable with appropriate initialization."""
    initial = tf.constant(0.1, shape = shape)
    #initial = tf.zeros(shape=shape)
    return tf.Variable(initial)

def variable_summaries(var):
    """Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
    with tf.name_scope('summaries'):
      mean = tf.reduce_mean(var)
      tf.summary.scalar('mean', mean)
      with tf.name_scope('stddev'):
        stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
      tf.summary.scalar('stddev', stddev)
      tf.summary.scalar('max', tf.reduce_max(var))
      tf.summary.scalar('min', tf.reduce_min(var))
      tf.summary.histogram('histogram', var)

def layer(input_tensor, input_dim, output_dim, layer_name, act=tf.nn.relu):
    # Adding a name scope ensures logical grouping of the layers in the graph.
    with tf.name_scope(layer_name):
        '''VARIABLES'''
        # These are the parameters that we are going to be training. The weight
        # matrix will be initialized using random values following a (truncated)
        # normal distribution. The biases get initialized to zero.

        # This Variable will hold the state of the weights for the layer
        with tf.name_scope('weights'):
            weights = weight_variable([input_dim, output_dim])
            variable_summaries(weights)

        with tf.name_scope('biases'):
            biases = bias_variable([output_dim])
            variable_summaries(biases)

        '''TRAINING COMPUTATION'''
        # We multiply the inputs with the weight matrix, and add biases. We compute
        # the softmax and cross-entropy (it's one operation in TensorFlow, because
        # it's very common, and it can be optimized)
        with tf.name_scope('Wx_plus_b'):
            logits = tf.matmul(input_tensor, weights) + biases
            tf.summary.histogram('logits', logits)

        return logits, weights, biases     
    
def accuracy(predictions, labels):
    with tf.name_scope('correct_prediction'):
      #correct_prediction = tf.equal(tf.argmax(predictions, axis=1), labels)
      correct_prediction = tf.equal(tf.math.argmax(predictions, axis=1), tf.math.argmax(labels, axis=1))
    with tf.name_scope('accuracy'):
        #accuracy = (100.0 * np.sum(correct_prediction) / predictions.shape[0])       
        accuracy = 100.0 * tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    tf.summary.scalar('accuracy', accuracy)
    return accuracy

# Create graph object: instantiate
graph = tf.Graph()

with graph.as_default():
    '''INPUT DATA'''
    # Load the training, validation and test data into constants that are
    # attached to the graph.
    # Input placeholders
    with tf.name_scope('input'):
        with tf.name_scope('Train'):
            x = tf.placeholder(tf.float32, shape = [None, image_size * image_size], name = 'x-input')
            y_ = tf.placeholder(tf.float32, shape = [None, num_labels], name = 'y-input')
        with tf.name_scope('Validation'):
            x_valid = tf.placeholder(tf.float32, shape = [None, image_size * image_size], name = 'x-input')
            y_valid = tf.placeholder(tf.float32, shape = [None, num_labels], name = 'y-input')

        with tf.name_scope('Test'):
            x_test = tf.placeholder(tf.float32, shape = [None, image_size * image_size], name = 'x-input')
            y_test = tf.placeholder(tf.float32, shape = [None, num_labels], name = 'y-input')
            
    # Save Image
    with tf.name_scope('input_reshape'):
       image_shaped_input = tf.reshape(x, [-1, 28, 28, 1])
       tf.summary.image('input', image_shaped_input, 10)
    
    ##Run Layer
    logits, weights, biases = layer(x, image_size * image_size, num_labels, "Layer")

    with tf.name_scope('cross_entropy'):
        # We take the average of this
        # cross-entropy across all training examples: that's our loss.
        with tf.name_scope('total'):
            loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels = y_, logits = logits))
    
    #Add to Graph
    tf.summary.scalar("loss", loss)

    with tf.name_scope('train'):
        '''OPTIMIZER'''
        # We are going to find the minimum of this loss using gradient descent.
        # 0.5 is the learning rate
        optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    '''PREDICTIONS for the training, validation, and test data.'''
    # These are not part of training, but merely here so that we can report
    # accuracy figures as we train.
    with tf.name_scope('Prediction'):
        train_prediction = tf.nn.softmax(logits)
    
    ##Accuracy
    tr_acc = accuracy(train_prediction, y_)
    
    with tf.name_scope('valid'):
        with tf.name_scope('Prediction'):
            with tf.name_scope('Wx_plus_b'):
                logits = tf.matmul(x_valid, weights) + biases
                tf.summary.histogram('logits', logits)
            valid_prediction = tf.nn.softmax(logits)
        v_acc = accuracy(valid_prediction, y_valid)
        
    with tf.name_scope('test'):
        with tf.name_scope('Prediction'):
            with tf.name_scope('Wx_plus_b'):
                logits = tf.matmul(x_test, weights) + biases
                tf.summary.histogram('logits', logits)
            test_prediction = tf.nn.softmax(logits)
        ts_acc = accuracy(test_prediction, y_test)
    
    accuracy = [tr_acc , v_acc, ts_acc]

Let's run this computation and iterate:

In [39]:
num_steps = 801

# Train the model, and also write summaries.
# All other steps, run train_step on training data, & add training summaries

def feed_dict():
    """Make a TensorFlow feed_dict: maps data onto Tensor placeholders."""
    xs = train_dataset[:train_subset, :]
    ys = train_labels[:train_subset]

    #Valid Data
    xv = valid_dataset
    yv = valid_labels
    
    #Test Data
    xt = test_dataset
    yt = test_labels

    return {x: xs, y_: ys, x_valid: xv, y_valid: yv, x_test: xt, y_test: yt}

def accuracyOld(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph = graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.global_variables_initializer().run()
  #session.run(tf.global_variables_initializer())
  
  # Merge all summary inforation.
  merged = tf.summary.merge_all()
  write = tf.summary.FileWriter(log_dir + "/multinomial", session.graph)
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    # Record summaries and test-set accuracy
    summary, l, acc, predictions, _ = session.run([merged, loss, accuracy, train_prediction, optimizer],
                                                  feed_dict=feed_dict())

    write.add_summary(summary, step)
    if (step % 100 == 0):
      print('======================================')
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.2f%%' % acc[0])
      print('(Old) Training accuracy: %.2f%%' % accuracyOld(predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.2f%%' % acc[1])
      print('(Old) Validation accuracy: %.2f%%' % accuracyOld(valid_prediction.eval({x_valid: valid_dataset}), valid_labels))
  print('=============== Finished!! =====================')  
  print('Test accuracy: %.2f%%' % acc[2])
  print('(Old) Test accuracy: %.2f%%' % accuracyOld(test_prediction.eval({x_test: test_dataset}), test_labels))
  write.close()

Initialized
Loss at step 0: 2.850958
Training accuracy: 11.10%
(Old) Training accuracy: 11.10%
Validation accuracy: 11.32%
(Old) Validation accuracy: 45.37%
Loss at step 100: 0.282835
Training accuracy: 94.40%
(Old) Training accuracy: 94.40%
Validation accuracy: 78.61%
(Old) Validation accuracy: 78.59%
Loss at step 200: 0.173570
Training accuracy: 97.40%
(Old) Training accuracy: 97.40%
Validation accuracy: 78.32%
(Old) Validation accuracy: 78.33%
Loss at step 300: 0.121441
Training accuracy: 98.90%
(Old) Training accuracy: 98.90%
Validation accuracy: 78.00%
(Old) Validation accuracy: 78.00%
Loss at step 400: 0.092088
Training accuracy: 99.70%
(Old) Training accuracy: 99.70%
Validation accuracy: 77.94%
(Old) Validation accuracy: 77.94%
Loss at step 500: 0.073730
Training accuracy: 99.80%
(Old) Training accuracy: 99.80%
Validation accuracy: 77.76%
(Old) Validation accuracy: 77.76%
Loss at step 600: 0.061319
Training accuracy: 99.80%
(Old) Training accuracy: 99.80%
Validation accuracy: 77

Let's now switch to stochastic gradient descent training instead, which is much faster.

The graph will be similar, except that instead of holding all the training data into a constant node, we create a `Placeholder` node which will be fed actual data at every call of `session.run()`.

In [43]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_train_labels, logits=logits))
  
  tf_summary_loss_SGD = tf.summary.scalar("loss-SGD", loss)
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Let's run it:

In [48]:
num_steps = 3001

def feed_dict_SGD(step):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    return {tf_train_dataset : batch_data, tf_train_labels : batch_labels}

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  
  # Merge all summary information.
  merged = tf.summary.merge_all()
  write = tf.summary.FileWriter(log_dir + "/SGD", session.graph)

  print("Initialized")
  for step in range(num_steps):

    summary, _, l, predictions = session.run(
      [merged, optimizer, loss, train_prediction], feed_dict=feed_dict_SGD(step))
    
    write.add_summary(summary, step)
    #accuracy = tf.Summary(value=[tf.Summary.Value(tag="training_accuracy", simple_value=accuracy(predictions, batch_labels))])
    #accuracy_summary.value[1].simple_value = accuracy(predictions, batch_labels)
    #write.add_summary(accuracy_summary, step)
    #tf_summary_loss_SGD = tf.summary.scalar("loss-SGD", loss)
    
    if (step % 500 == 0):
      print('======================================')
      print("Minibatch loss at step %d: %f" % (step, l))
      #print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      #print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print('=================Finished!!=====================')  
  #print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 14.876855
Minibatch loss at step 500: 1.480255
Minibatch loss at step 1000: 1.185415
Minibatch loss at step 1500: 0.971577
Minibatch loss at step 2000: 0.982563
Minibatch loss at step 2500: 0.759568
Minibatch loss at step 3000: 1.107866
=================Finished!!=====================


---
Problem
-------

Turn the logistic regression example with SGD into a 1-hidden layer neural network with rectified linear units [nn.relu()](https://www.tensorflow.org/versions/r0.7/api_docs/python/nn.html#relu) and 1024 hidden nodes. This model should improve your validation / test accuracy.

---

In [ ]:
num_nodes= 1024
batch_size = 128

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    weights_1 = tf.Variable(
        tf.truncated_normal([image_size * image_size, num_nodes]))
    biases_1 = tf.Variable(tf.zeros([num_nodes]))
    weights_2 = tf.Variable(tf.truncated_normal([num_nodes, num_labels]))
    biases_2 = tf.Variable(tf.zeros([num_labels]))
    
    logits_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
    relu_layer= tf.nn.relu(logits_1)
    logits_2 = tf.matmul(relu_layer, weights_2) + biases_2
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_train_labels, logits=logits_2))
    
    tf_summary_loss_SGD_1h = tf.summary.scalar("loss-SGD-1-hidden", loss)
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training
    train_prediction = tf.nn.softmax(logits_2)
    
    # Predictions for validation 
    logits_1 = tf.matmul(tf_valid_dataset, weights_1) + biases_1
    relu_layer= tf.nn.relu(logits_1)
    logits_2 = tf.matmul(relu_layer, weights_2) + biases_2
    
    valid_prediction = tf.nn.softmax(logits_2)
    
    # Predictions for test
    logits_1 = tf.matmul(tf_test_dataset, weights_1) + biases_1
    relu_layer= tf.nn.relu(logits_1)
    logits_2 = tf.matmul(relu_layer, weights_2) + biases_2
    
    test_prediction =  tf.nn.softmax(logits_2)

In [ ]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    # Merge all summary inforation.
    summary_merge = tf.summary.merge_all()
    write.add_graph(session.graph, num_steps)
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        summary, _, l, predictions = session.run([summary_merge, optimizer, loss, train_prediction], feed_dict=feed_dict)
        write.add_summary(summary, step)

        #accuracy = tf.Summary(value=[tf.Summary.Value(tag="training_accuracy", simple_value=accuracy(predictions, batch_labels))])
        accuracy_summary.value[2].simple_value = accuracy(predictions, batch_labels)
        write.add_summary(accuracy_summary, step)
        
        #tf_summary_loss_SGD_1h = tf.summary.scalar("loss-SGD-1-hidden", loss)
        if (step % 500 == 0):
          print('======================================')
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy(
              valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

In [ ]:
performance_summaries = tf.summary.merge([tf_summary_loss, tf_summary_loss_SGD, tf_summary_loss_SGD_1h])
write.add_summary(performance_summaries)